In [0]:
import tensorflow as tf

In [0]:
layers_in_block = {
    'resnet-18' : [2, 2, 2, 2],
    'resnet-34' : [3, 4, 6, 3],
    'resnet-50' : [3, 4, 6, 3],
    'resnet-101' : [3, 4, 23, 3],
    'resnet-152' : [3, 8, 36, 3]
    }

class ResNet_Dense(tf.keras.layers.Layer):
  def __init__(self, pooling='avg', classes=1):
    super(ResNet_Dense, self).__init__()
    self.classes = classes
    self.pooling = pooling

    self.avg_pooling = tf.keras.layers.GlobalAveragePooling2D()
    self.max_pooling = tf.keras.layers.GlobalMaxPooling2D()

    self.sigmoid_fc = tf.keras.layers.Dense(units=classes, activation=tf.keras.activations.sigmoid)
    self.softmax_fc = tf.keras.layers.Dense(units=classes, activation=tf.keras.activations.softmax)


  def call(self, inputs):
    if self.pooling == 'avg':
      x = self.avg_pooling(inputs)
    elif self.pooling == 'max':
      x = self.max_pooling(inputs)

    if self.classes == 1:
      x = self.sigmoid_fc(x)
    else:
      x = self.softmax_fc(x)
      
    return x

def make_dense_layer(pooling='avg', classes=1):
  res_dense = tf.keras.Sequential()
  res_dense.add(ResNet_Dense(pooling=pooling, classes=classes))
  return res_dense

## dimension을 맞추기 위해 stride 2 설정(basic, bottle commit)

In [0]:
class Basic_building_block(tf.keras.layers.Layer):
  def __init__(self, filter_num=None, kernel_size=(3, 3), \
               stride=1, padding='same'):
    super(Basic_building_block, self).__init__()
    
    #First layer of the block
    self.conv1 = tf.keras.layers.Conv2D(filters=filter_num,
                                       kernel_size=kernel_size,
                                       strides=stride,
                                       padding=padding)
    self.bn1 = tf.keras.layers.BatchNormalization()

    #Second layer of the block
    self.conv2 = tf.keras.layers.Conv2D(filters=filter_num,
                                        kernel_size=kernel_size,
                                        strides=1,
                                        padding=padding)
    self.bn2 = tf.keras.layers.BatchNormalization()

    if stride != 1:
      self.downsample = tf.keras.Sequential()
      self.downsample.add(tf.keras.layers.Conv2D(filters=filter_num,
                                                  kernel_size=(1, 1),
                                                  strides=stride))
      self.downsample.add(tf.keras.layers.BatchNormalization())
    else:
      self.downsample = lambda x: x

  def call(self, inputs, training=None):
    x_shortcut = self.downsample(inputs)

    x = self.conv1(inputs)
    x = self.bn1(x, training=training)
    x = tf.nn.relu(x)
    x = self.conv2(x)
    x = self.bn2(x, training=training)
    x = tf.nn.relu(tf.keras.layers.add([x_shortcut, x]))
    
    return x

def make_basic_block_layer(filter_num, blocks, stride=1):
  res_block = tf.keras.Sequential()
  for i in range(blocks):
    if stride==2 and i == 0:
      res_block.add(Basic_building_block(filter_num, stride=stride))
    else:
      res_block.add(Basic_building_block(filter_num, stride=1))
  return res_block

class BasicResNet(tf.keras.Model):
  def __init__(self, layer='resnet-34'):
    super(BasicResNet, self).__init__()

    self.conv1 = tf.keras.layers.Conv2D(filters=64, kernel_size=(7, 7), strides=2, padding='same', kernel_initializer='he_normal')
    self.bn1 = tf.keras.layers.BatchNormalization(axis=3)
    self.max_pooling = tf.keras.layers.MaxPooling2D(pool_size=(3, 3), strides=2)

    self.res_block1 = make_basic_block_layer(64, layers_in_block[layer][0], 1)
    self.res_block2 = make_basic_block_layer(128, layers_in_block[layer][1], 2)
    self.res_block3 = make_basic_block_layer(256, layers_in_block[layer][2], 2)
    self.res_block4 = make_basic_block_layer(512, layers_in_block[layer][3], 2)

    self.fc_block1 = make_dense_layer('avg', 1)
    
  def call(self, inputs, training=None):
    x = self.conv1(inputs)
    x = self.bn1(x, training=training)
    x = tf.nn.relu(x)
    x = self.max_pooling(x)

    x = self.res_block1(x)
    x = self.res_block2(x)
    x = self.res_block3(x)
    x = self.res_block4(x)

    x = self.fc_block1(x)
    
    return x

In [0]:
class Bottle_building_block(tf.keras.layers.Layer):
    def __init__(self, filter_num=None, stride=2):
      super(Bottle_building_block, self).__init__()
      #First layer of the block
      self.conv1 = tf.keras.layers.Conv2D(filters=filter_num,
                                        kernel_size=(1, 1),
                                        strides=1,
                                        padding='valid')
      self.bn1 = tf.keras.layers.BatchNormalization()

      #Second layer of the block
      self.conv2 = tf.keras.layers.Conv2D(filters=filter_num,
                                          kernel_size=(3, 3),
                                          strides=stride,
                                          padding='same')
      self.bn2 = tf.keras.layers.BatchNormalization()

      #Three layer of the block
      self.conv3 = tf.keras.layers.Conv2D(filters=filter_num*4,
                                          kernel_size=(1, 1),
                                          strides=1,
                                          padding='valid')
      self.bn3 = tf.keras.layers.BatchNormalization()

      self.downsample = tf.keras.Sequential()
      self.downsample.add(tf.keras.layers.Conv2D(filters=filter_num*4,
                                                kernel_size=(1, 1),
                                                strides=stride,
                                                padding='valid'))
      self.downsample.add(tf.keras.layers.BatchNormalization())
    
    def call(self, inputs, training=None):
      x_shortcut = self.downsample(inputs)

      x = self.conv1(inputs)
      x = self.bn1(x, training=training)
      x = tf.nn.relu(x)
      x = self.conv2(x)
      x = self.bn2(x, training=training)
      x = tf.nn.relu(x)
      x = self.conv3(x)
      x = self.bn3(x, training=training)

      x = tf.nn.relu(tf.keras.layers.add([x_shortcut, x]))
  
      return x

def make_bottle_block_layer(filter_num, blocks, stride=2):
  res_block = tf.keras.Sequential()
  for i in range(blocks):
    if i == 0:
      res_block.add(Bottle_building_block(filter_num, stride=stride))
    else:
      res_block.add(Bottle_building_block(filter_num, stride=1))
  return res_block

class BottleResNet(tf.keras.Model):
  def __init__(self, layer='resnet-101'):
    super(BottleResNet, self).__init__()

    self.zero_padd_1 = tf.keras.layers.ZeroPadding2D(padding=(3, 3))
    self.conv1 = tf.keras.layers.Conv2D(filters=64, 
                                        kernel_size=(7, 7), 
                                        strides=2)
    self.bn1 = tf.keras.layers.BatchNormalization()
    self.zero_padd_2 = tf.keras.layers.ZeroPadding2D(padding=(1, 1))
    self.max_pooling = tf.keras.layers.MaxPooling2D(pool_size=(3, 3), 
                                                    strides=2)

    self.res_block1 = make_bottle_block_layer(64, layers_in_block[layer][0], 1)
    self.res_block2 = make_bottle_block_layer(128, layers_in_block[layer][1], 2)
    self.res_block3 = make_bottle_block_layer(256, layers_in_block[layer][2], 2)
    self.res_block4 = make_bottle_block_layer(512, layers_in_block[layer][3], 2)

    self.fc_block1 = make_dense_layer('avg', 1)
    
  def call(self, inputs, training=None):
    x = self.zero_padd_1(inputs)
    x = self.conv1(x)
    x = self.bn1(x, training=training)
    x = tf.nn.relu(x)
    x = self.zero_padd_2(x)
    x = self.max_pooling(x)

    x = self.res_block1(x)
    x = self.res_block2(x)
    x = self.res_block3(x)
    x = self.res_block4(x)

    x = self.fc_block1(x)

    return x

In [0]:
def resnet_18():
  return BasicResNet(layer='resnet-18')

def resnet_34():
  return BasicResNet(layer='resnet-34')

def resnet_50():
  return BottleResNet(layer='resnet-50')

def resnet_101():
  return BottleResNet(layer='resnet-101')

def resnet_152():
  return BottleResNet(layer='resnet-152')